# Exploring the senegalese tax-benefit-system with OpenFisca

In [1]:
!pip install matplotlib
!pip install openfisca-senegal[survey]

     |████████████████████████████████| 108 kB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 39.3 MB 47.6 MB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 115.7 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 14.2 MB/s  eta 0:00:01
  Using cached tables-3.6.1-cp38-cp38-manylinux1_x86_64.whl (4.3 MB)
  Using cached tabulate-0.8.9-py3-none-any.whl (25 kB)
Processing /home/mbenjelloul/.cache/pip/wheels/b5/d3/ae/700722865a86d3de95f2d04144d4560d75699797bca5353411/weightedcalcs-0.1.2-py3-none-any.whl
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached wquantiles-0.6-py3-none-any.whl (3.3 kB)
  Using cached pyxdg-0.27-py2.py3-none-any.whl (49 kB)


In [ ]:
import matplotlib.pyplot as plt  # For graphics
%matplotlib inline

from openfisca_core import periods

from openfisca_senegal import CountryTaxBenefitSystem as SenegalTaxBenefitSystem  # The Senegalese tax-benefits system
from openfisca_senegal.scenarios import init_single_entity

%config Completer.use_jedi = False


## The concepts

### Tax Benefit System

In [ ]:
tax_benefit_system = SenegalTaxBenefitSystem()

### Entities

In [ ]:
tax_benefit_system.entities

In [ ]:
[entity.key for entity in tax_benefit_system.entities]

In [ ]:
tax_benefit_system.entities

### Variables 

In [ ]:
tax_benefit_system.variables

In [ ]:
sorted(tax_benefit_system.variables.keys())

In [ ]:
import inspect
for name, variable in sorted(tax_benefit_system.variables.items()):
    print(name)
    if not variable.is_input_variable():
        formula = variable.get_formula('2017')
        source_code = inspect.getsourcelines(formula)[0]
        print(''.join(source_code))
    print('')

### Parameters

In [ ]:
print(tax_benefit_system.parameters)

In [ ]:
parameters = tax_benefit_system.parameters(2017)

In [ ]:
parameters.prelevements_obligatoires.impots_directs.bareme_impot_progressif

## A simple test case scenario

### Basics

In [ ]:
scenario = tax_benefit_system.new_scenario()
scenario

In [ ]:
init_single_entity(
    scenario,
    parent1 = {
        'salaire': 2800000,
        'est_marie': True,
        'conjoint_a_des_revenus': False,
        'nombre_enfants': 1,
        },
    period = '2017',
    )

In [ ]:
simulation = scenario.new_simulation()

In [ ]:
simulation.calculate('salaire', period = '2017')

In [ ]:
simulation.calculate('impot_revenus', period = '2017')

### Application: a simple function to compute the tax gain for having a child

In [ ]:
def tax_gain(salaire = 0, est_marie = False, conjoint_a_des_revenus = False, 
             nombre_enfants = 0):
    period = '2017'
    parent1_before = {
        'salaire': salaire,
        'est_marie': est_marie,
        'conjoint_a_des_revenus': conjoint_a_des_revenus,
        'nombre_enfants': nombre_enfants,
        }
    simulation_before = init_single_entity(
        tax_benefit_system.new_scenario(),
        parent1 = parent1_before,
        period = period,
        ).new_simulation()
    parent1_after = parent1_before.copy()
    parent1_after['nombre_enfants'] += 1
    simulation_after = init_single_entity(
        tax_benefit_system.new_scenario(),
        parent1 = parent1_after,
        period = period,
        ).new_simulation()
    difference = simulation_after.calculate('impot_revenus', period) - simulation_before.calculate('impot_revenus', period)
    return -difference
                                                                                            

In [ ]:
tax_gain(salaire = 2800000)

In [ ]:
tax_gain(salaire = 2800000, nombre_enfants = 2)

In [ ]:
[tax_gain(salaire = 2800000, nombre_enfants = n) for n in range(8)]

## More complex scenario: varying revenue 

### Basics

In [ ]:
year = 2017
simulation = init_single_entity(
    tax_benefit_system.new_scenario(),
    parent1 = {
        'est_marie': True,
        'conjoint_a_des_revenus': False,
        'nombre_enfants': 0,
        },
    period = year,
    axes = [[
        {
            'count': 100,
            'min': 0,
            'max': 15e6,
            'name': 'salaire',
            }
        ]],
    ).new_simulation()
salaire = simulation.calculate('salaire', period = year)
reduction_impots_pour_charge_famille = simulation.calculate('reduction_impots_pour_charge_famille', period = year)
impot_avant_reduction_famille = simulation.calculate('impot_avant_reduction_famille', period = year)
impot_revenus = simulation.calculate('impot_revenus', period = year)

In [ ]:
len(reduction_impots_pour_charge_famille)

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(salaire, impot_avant_reduction_famille, label='avant réduction famille')
plt.plot(salaire, reduction_impots_pour_charge_famille, label='réduction famille')
plt.plot(salaire, impot_revenus, label='impôt revenus')
plt.xlabel('Salaire')
plt.legend()
plt.title('0 enfants', fontsize=20)

### Application: by how much the tax gain for having a child varies with revenue

#### Write the function `tax_gain_varying_revenue` that computes the gain of having a child as a function of the revenue

In [ ]:
def tax_gain_varying_revenue(max_salaire = 0, est_marie = False, conjoint_a_des_revenus = False, 
                             nombre_enfants = 0):
    TODO
    return salaire, -difference

In [ ]:
salaire, gain = tax_gain_varying_revenue(max_salaire = 15e6)

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(salaire, gain, label='Gain fiscal')
plt.xlabel('Salaire')
plt.legend()
plt.title('Gain fiscal au 1er enfant', fontsize=20)

#### Write a loop to plot the gain for several children

In [ ]:
plt.figure(figsize=(12, 8))
TODO
plt.xlabel('Salaire')
plt.legend()
plt.title("Gain fiscal selon le rang de l'enfant", fontsize=20)


## Tax rates

### Average tax rate

In [ ]:
simulation = init_single_entity(
    tax_benefit_system.new_scenario(),
    parent1 = {
        'est_marie': True,
        'conjoint_a_des_revenus': False,
        'nombre_enfants': 0,
        },
    period = year,
    axes = [[
        {
            'count': 100,
            'min': 0,
            'max': 18e6,
            'name': 'salaire',
            }
        ]],
    ).new_simulation()
salaire = simulation.calculate('salaire', period = year)
impot = simulation.calculate('impot_revenus', period = year)

In [ ]:
plt.plot(salaire, impot)
plt.ylabel(u"Impôt sur les revenus")
plt.xlabel(u"Salaire")

In [ ]:
taux_moyen = impot / salaire  # Average tax rate

In [ ]:
taux_moyen = impot / (salaire * (salaire != 0) + (salaire == 0) )  

In [ ]:
plt.plot(salaire, taux_moyen)
plt.ylabel("Taux moyen")
plt.xlabel("Salaire")

### Marginal tax rate

In [ ]:
taux_marginal =  (impot[:-1] - impot[1:]) / (salaire[:-1] - salaire[1:] )  # We avoid the first point

In [ ]:
plt.plot(salaire[:-1], taux_marginal)

#### Exercice: how marginal and average tax rate compare at infinity

### Add a new variable and use dedicated functions

Add a disposable income variable to the tax-benefit-system

In [ ]:
from openfisca_core.model_api import *
from openfisca_senegal.entities import Person

tax_benefit_system = SenegalTaxBenefitSystem()

class revenu_disponible(Variable):
    definition_period = YEAR
    entity = Person
    value_type = float

    def formula(person, period):
        salaire = person('salaire', period)
        impot_revenus = person('impot_revenus', period)
        return salaire - impot_revenus


tax_benefit_system.add_variable(revenu_disponible)


In [ ]:
simulation = init_single_entity(
    tax_benefit_system.new_scenario(),
    parent1 = {
        'est_marie': True,
        'conjoint_a_des_revenus': False,
        'nombre_enfants': 0,
        },
    period = '2017',
    axes = [[
        {
            'count': 100,
            'min': 0,
            'max': 18e6,
            'name': 'salaire',
            }
        ]],
    ).new_simulation()
salaire = simulation.calculate('salaire', period = year)
impot = simulation.calculate('impot_revenus', period = year)
revenu_disponible = simulation.calculate('revenu_disponible', period = year)

In [ ]:
from openfisca_core.rates import average_rate, marginal_rate
plt.plot(salaire[1:], average_rate(revenu_disponible[1:], salaire[1:]))
plt.plot(salaire[1:], marginal_rate(revenu_disponible, salaire))

## Reforming the tax-benefit system

### Parametric reform

In [ ]:
def modify_parameters(parameters):
    parameters.prelevements_obligatoires.impots_directs.bareme_impot_progressif[5].rate.update(period = period(year), value = .5)
    return parameters
    
class tax_the_rich(Reform):
    name = u"Tax last bracket at 50%"

    def apply(self):
        self.modify_parameters(modifier_function = modify_parameters)

In [ ]:
reform = tax_the_rich(tax_benefit_system)

In [ ]:
simulation = init_single_entity(
    reform.new_scenario(),
    parent1 = {
        'est_marie': True,
        'conjoint_a_des_revenus': False,
        'nombre_enfants': 0,
        },
    period = year,
    axes = [[
        {
            'count': 100,
            'min': 0,
            'max': 18e6,
            'name': 'salaire',
            }
        ]],
    ).new_simulation()
salaire = simulation.calculate('salaire', period = year)
impot = simulation.calculate('impot_revenus', period = year)
taux_marginal =  (impot[:-1] - impot[1:]) / (salaire[:-1] - salaire[1:] )  # We avoid the first point
plt.plot(salaire[:-1], taux_marginal)

### Structural Reform

#### Exercice: add a an amount threshold under which no tax is due

In [ ]:
def build_structural_reform(seuil = 10000):    
    tax_benefit_system = SenegalTaxBenefitSystem()
    
    class impot_revenus(Variable):
        def formula(individu, period):
            # TODO rewrite a new impot_revenus wich have a non-payment threshold (seuil)
            
    class revenu_disponible(Variable):
        definition_period = YEAR
        entity = Person
        value_type = float

        def formula(person, period):
            salaire = person('salaire', period)
            impot_revenus = person('impot_revenus', period)
            return salaire - impot_revenus

    class structural_reform(Reform):
        name = u"Seuil de non-versement: {}".format(seuil)

        def apply(self):
            self.update_variable(impot_revenus)
            self.add_variable(revenu_disponible)

    return structural_reform(tax_benefit_system)

In [ ]:
structural_reform = build_structural_reform(seuil = 100000)
simulation = init_single_entity(
    structural_reform.new_scenario(),
    parent1 = {
        'est_marie': True,
        'conjoint_a_des_revenus': False,
        'nombre_enfants': 0,
        },
    period = '2017',
    axes = [[
        {
            'count': 100,
            'min': 0,
            'max': 18e6,
            'name': 'salaire',
            }
        ]],
    ).new_simulation()
salaire = simulation.calculate('salaire', period = year)
impot_verse = simulation.calculate('impot_revenus', period = year)
revenu_disponible = simulation.calculate('revenu_disponible', period = year)

In [ ]:
plt.plot(salaire, impot_verse)
plt.ylabel(u"Impôt sur les revenus")
plt.xlabel(u"Salaire")

In [ ]:
plt.plot(salaire[1:], average_rate(revenu_disponible[1:], salaire[1:]))
plt.plot(salaire[1:], marginal_rate(revenu_disponible, salaire))